In [1]:
%load_ext autoreload

In [21]:
from pathlib import Path

import altair as alt
import polars as pl

%autoreload
from altair_utils import (
    color_sf_bayarea,
    color_sfcore_restofsf_restofbayarea,
    line_strokeDash,
)

In [3]:
def parse_csv(df):
    return (
        df.filter((pl.col("year") > 2016))
        .with_columns(
            geography=pl.when("northeast_core")
            .then(pl.lit("northeast core"))
            .otherwise(pl.lit("rest of SF"))
        )
        .drop("northeast_core")
    )

In [4]:
dir = Path(r"Q:\Model Research\downtown_today\cmp-stats")
auto_arterial = (
    parse_csv(pl.read_csv(dir / "cmp_segments-auto-speed.csv"))
    .filter((pl.col("inrix_segments") == "XD") & (pl.col("fac_typ") == "Arterial"))
    .drop("inrix_segments")
)
transit = parse_csv(pl.read_csv(dir / "cmp_segments-transit-speed.csv"))
geog_color = alt.Color("geography").scale(
    domain=["northeast core", "rest of SF"],
    range=color_sfcore_restofsf_restofbayarea[:2],
)

In [5]:
auto_freeway_path = r"Q:\CMP\LOS Monitoring 2023\cmp_aggregate_xd.csv"
auto_freeway = (
    pl.read_csv(auto_freeway_path)
    .filter((pl.col("viz") == "ALOS") & (pl.col("fac_typ") == "Freeway"))
    .drop("viz")
    .rename({"metric": "speed"})
)

In [ ]:
auto_freeway_chart = (
    alt.Chart(auto_freeway)
    .mark_line()
    .encode(
        x="year:O",
        y=alt.Y("speed", title="CMP network (freeways) auto speeds"),
        color=alt.value(color_sf_bayarea[0]),
        strokeDash=alt.StrokeDash("period").scale(
            domain=["AM", "PM"],
            range=line_strokeDash,
        ),
        tooltip=["year", "period", "speed"],
    )
)
auto_freeway.write_csv("output/data/cmp_network-freeway-auto-speed.csv")
auto_freeway_chart.properties(width=200).save(
    "output/Links/cmp_network-freeway-auto-speed.png", scale=3
)
auto_freeway_chart.properties(width="container").save(
    "output/Links/cmp_network-freeway-auto-speed.html"
)
auto_freeway_chart.properties(width=200).interactive()

In [ ]:
auto_arterial_chart = (
    alt.Chart(auto_arterial)
    .mark_line()
    .encode(
        x="year:O",
        y=alt.Y("speed", title="CMP network (arterials) auto speeds"),
        color=geog_color,
        strokeDash=alt.StrokeDash("period").scale(
            domain=["AM", "PM"],
            range=line_strokeDash,
        ),
        tooltip=["year", "period", "geography", "speed"],
    )
)
auto_arterial_chart.properties(width=200).save(
    "output/Links/cmp_network-arterial-auto-speed.png", scale=3
)
auto_arterial_chart.properties(width="container").save(
    "output/Links/cmp_network-arterial-auto-speed.html"
)
auto_arterial_chart.properties(width=200)

In [ ]:
transit_chart = (
    alt.Chart(
        transit.filter((pl.col("fac_typ") == "Arterial") & (pl.col("year") > 2016)),
        title="CMP network (arterials) transit speeds",
    )
    .mark_line()
    .encode(
        x="year:O",
        y="speed",
        color=geog_color,
        strokeDash=alt.StrokeDash("period").scale(
            domain=["AM", "PM"],
            range=line_strokeDash,
        ),
        tooltip=["year", "period", "geography", "speed"],
    )
)
transit_chart.properties(width=200).save(
    "output/Links/cmp_network-arterial-transit-speed.png", scale=3
)
transit_chart.properties(width="container").save(
    "output/Links/cmp_network-arterial-transit-speed.html"
)
transit_chart.properties(width=200)